In [1]:
import pandas as pd

import json
import numpy as np  # Предполагаем, что используется NumPy
from tqdm.auto import tqdm
import requests
from io import BytesIO
from PIL import Image, ExifTags
import os


import json
import numpy as np
from tqdm.auto import tqdm


import re


def extract_ip_adapter_img(item):

    # Example string containing URLs
    big_string = str(item)

    # Regular expression pattern to match the specific URLs
    pattern = r'https://cdn\.lovescape\.com/cdn/photos/[a-zA-Z0-9/.-]+'

    # Find all matches in the big string
    matches = re.findall(pattern, big_string)
    if matches:
        return matches[0]
    else:
        return None

from urllib.parse import urlparse, parse_qs
import time

def is_url_expired(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    
    if 'Expires' in query_params:
        expires_timestamp = int(query_params['Expires'][0])
        current_timestamp = int(time.time())
        return current_timestamp < expires_timestamp
    else:
        raise ValueError("The URL does not contain an 'Expires' parameter.")

def find_key_in_nested_dict(d, target_key, simple_mode=True, return_all=False):
    result_data = []
    for d_item in d:
        data = _find_key_in_nested_dict(d_item, target_key, simple_mode=simple_mode)
        if data:
            result_data.extend(data)
    if len(result_data) >= 1 and not return_all:
        return result_data[0]
    elif len(result_data) >= 1 and return_all:
        return result_data
    else:
        return None

def extract_ip_adapter_img(item):

    # Example string containing URLs
    big_string = str(item)

    # Regular expression pattern to match the specific URLs
    pattern = r'https://cdn\.lovescape\.com/cdn/photos/[a-zA-Z0-9/.-]+'

    # Find all matches in the big string
    matches = re.findall(pattern, big_string)
    if matches:
        return matches[0]
    else:
        return None

def contains_pattern(string):
    # Regular expression to match substrings like __some-text__
    pattern = r'__[^_]+__'
    # Use re.search to check if the pattern exists in the string
    return not(bool(re.search(pattern, string)))


def recursive_merge(dict1, dict2):
    """
    Recursively merges dict2 into dict1.
    - If a key exists in both dictionaries and the values are also dictionaries, merge them recursively.
    - If a key exists in both dictionaries but the values are not dictionaries, overwrite the value in dict1 with the value from dict2.
    - If a key exists only in dict2, add it to dict1.

    Args:
        dict1 (dict): The base dictionary to merge into.
        dict2 (dict): The dictionary to merge from.

    Returns:
        dict: The merged dictionary (modified dict1).
    """
    for key, value in dict2.items():
        if key in dict1:
            if isinstance(dict1[key], dict) and isinstance(value, dict):
                # Both values are dictionaries, merge them recursively
                recursive_merge(dict1[key], value)
            elif isinstance(dict1[key], list) and isinstance(value, list):
                # Both values are lists, extend the list
                dict1[key].extend(value)
            else:
                # Overwrite the value in dict1 with the value from dict2
                dict1[key] = value
        else:
            # Key does not exist in dict1, add it
            dict1[key] = value
    return dict1


def merge_json_from_numpy_list(numpy_list):
    json_objects = [] 
    broken_strings = 0
    fragments = {} 

    broken_strings = 0
    # 1. Parse JSON strings into Python dictionaries
    for json_string in tqdm(numpy_list, desc="Parsing JSON"):
        try:
            json_object = json.loads(json_string)
            json_objects.append(json_object)
            correlation_id = _find_key_in_nested_dict(json_object, 'correlation_id')
            correlation_id = [item['str'] for item in correlation_id if 'str' if item]
            correlation_id = correlation_id[0]
            if correlation_id not in fragments:
                fragments[correlation_id] = []
            fragments[correlation_id].append(json_object)
        except:
            broken_strings += 1

    return fragments


import json
from itertools import chain


def find_key_in_nested_dict(d, target_key, simple_mode=True, return_all=False):
    """
    Wrapper function to find a key in a nested dictionary or list of dictionaries.
    """
    if isinstance(d, list):
        # Use generator to avoid creating intermediate lists when not needed
        results = chain.from_iterable(
            _find_key_in_nested_dict(d_item, target_key, simple_mode=simple_mode) for d_item in d
        )
    else:
        results = _find_key_in_nested_dict(d, target_key, simple_mode=simple_mode)

    if return_all:
        return list(results)  # Collect all results
    return next(results, None)  # Return the first result or None if no match


def _find_key_in_nested_dict(d, target_key, simple_mode=True):
    """
    Recursively searches for a key in a nested dictionary or list, yielding results lazily.
    """
    # Ensure the input is a dictionary
    if not isinstance(d, dict):
        return

    for key, value in d.items():
        if key == target_key:
            if simple_mode and isinstance(value, dict) and len(value) == 1:
                yield value
            elif not simple_mode:
                yield value

        # If value is a string, try parsing it as JSON
        if isinstance(value, str):
            try:
                value = json.loads(value)
            except json.JSONDecodeError:
                pass  # Ignore if not a JSON string

        # Recurse into nested dictionaries or lists
        if isinstance(value, dict):
            yield from _find_key_in_nested_dict(value, target_key, simple_mode=simple_mode)
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    yield from _find_key_in_nested_dict(item, target_key, simple_mode=simple_mode)
                elif isinstance(item, str):  # Try parsing strings in lists as JSON
                    try:
                        item = json.loads(item)
                        if isinstance(item, dict):
                            yield from _find_key_in_nested_dict(item, target_key, simple_mode=simple_mode)
                    except json.JSONDecodeError:
                        pass





In [2]:
df = pd.read_csv("dataframes/new_data_04_12_2025.csv")
# Пример использования
numpy_list = np.array(df.message)
restored_objects = merge_json_from_numpy_list(numpy_list)

Parsing JSON:   0%|          | 0/1233727 [00:00<?, ?it/s]

In [ ]:
def unwrap_values(data):
    # Если это словарь
    if isinstance(data, dict):
        result = {}
        for key, value in data.items():
            # Проверяем, если это словарь с одним ключом, который указывает на тип
            if isinstance(value, dict) and len(value) == 1 and next(iter(value)) in {'int', 'str', 'float', 'bool', 'array', 'object'}:
                result[key] = unwrap_values(next(iter(value.values())))  # Берем значение этого ключа и обрабатываем его
            # Если значение - это словарь или список, обрабатываем рекурсивно
            else:
                result[key] = unwrap_values(value)
        return result

    # Если это список, обрабатываем каждый элемент
    elif isinstance(data, list):
        return [unwrap_values(item) for item in data]

    # Если это не словарь и не список, возвращаем как есть
    return data


# Обертка, чтобы избавиться от верхнего уровня 'object'
def unwrap_top_level(data):
    # Проверяем, если верхний уровень - словарь с ключом 'object'
    if isinstance(data, dict) and 'object' in data and isinstance(data['object'], dict):
        return unwrap_values(data['object'])  # Возвращаем только содержимое 'object'
    return unwrap_values(data)  # Если нет 'object', просто разворачиваем


def is_webui_config(item):
    if len(item) == 1:
        keys = list(item.keys())
        return is_webui_config(item[keys[0]])
    elif all(value in item for value in ['prompt', 'negative_prompt', 'seed', 'override_settings']):
        return True
    else:
        return False

def extract_webui_config(item):
    possible_webui_configs = [*find_key_in_nested_dict(item,'config', return_all=True), *find_key_in_nested_dict(item, 'payload', return_all=True)]
    possible_webui_configs = [unwrap_top_level(item) for item in possible_webui_configs]
    possible_webui_configs = list(filter(is_webui_config, possible_webui_configs))
    if possible_webui_configs:
        return possible_webui_configs[0]


def extract_usefull_data(item):
    """
    Extracts useful data from a nested dictionary.
    """
    def safe_get(key, default=None, return_all=False, simple_mode=True):
        result = find_key_in_nested_dict(item, key, simple_mode=simple_mode, return_all=return_all)
        return result if result is not None else default

    # result_image_url = safe_get('result_image_urls', {}).get('array', [None])[0]
    is_sfw = safe_get('sfw', {}).get('bool', None)
    composition = safe_get('composition', {}).get('str', None)
    style = safe_get('style', {'str': 'simple'}).get('str', 'simple')
    project = safe_get('project', {}).get('str', None)
    gender = safe_get('gender', {}).get('str', None)

    ip_adapter_img = extract_ip_adapter_img(item)  # Assuming this function is implemented elsewhere
    time_stamp = safe_get('datetime', {}).get('str', None)
    return {
        # "generation_time": generation_time,
        # "result_image": result_image_url,
        'webui_config': extract_webui_config(item),
        "ip_adapter_image": ip_adapter_img,
        "is_sfw": is_sfw,
        "time_stamp": time_stamp,
        "composition": composition,
        "style": style,
        "project": project,
        "gender": gender,
    }  


# def extract_usefull_data(item):
#     """
#     Extracts useful data from a nested dictionary.
#     """
#     def safe_get(key, default=None, return_all=False, simple_mode=True):
#         result = find_key_in_nested_dict(item, key, simple_mode=simple_mode, return_all=return_all)
#         return result if result is not None else default

#     result_image_url = safe_get('result_image_urls', {}).get('array', [None])[0]
#     is_sfw = safe_get('sfw', {}).get('bool', None)
#     composition = safe_get('composition', {}).get('str', None)
#     # checkpoint_name = safe_get('sd_model_checkpoint', {}).get('str', None)
#     style = safe_get('style', {'str': 'simple'}).get('str', 'simple')
#     project = safe_get('project', {}).get('str', None)
#     gender = safe_get('gender', {}).get('str', None)
#     # negative_prompt = safe_get('negative_prompt', {}).get('str', None)
#     # positive_prompts_all = find_key_in_nested_dict(item, 'prompt', return_all=True)
#     # positive_prompts_all = [item['str'] for item in positive_prompts_all if 'str' in item]
#     # positive_prompt = list(filter(contains_pattern, positive_prompts_all))
#     # if len(positive_prompt) == 0:
#     #     positive_prompt = min(positive_prompts_all, key=len)
#     # else:
#     #     positive_prompt = positive_prompts_all[0]

#     ip_adapter_img = extract_ip_adapter_img(item)  # Assuming this function is implemented elsewhere
#     time_stamp = safe_get('datetime', {}).get('str', None)
#     generation_time = safe_get('elapsed', {}).get('float', None)
#     width = safe_get('width', {'int': 768}).get('int', 768)
#     height = safe_get('height', {'int': 1152}).get('int', 1152)

#     return {
#         "generation_time": generation_time,
#         "result_image": result_image_url,
#         "ip_adapter_image": ip_adapter_img,
#         "is_sfw": is_sfw,
#         "time_stamp": time_stamp,
#         "composition": composition,
#         # "checkpoint_name": checkpoint_name,
#         "style": style,
#         "project": project,
#         "gender": gender,
#         # "positive_prompt": positive_prompt,
#         # "negative_prompt": negative_prompt,
#         "width": width,
#         "height": height,
#     }  

usefull_data = []
lost_data_count = 0
lost_data = []

for key in tqdm(restored_objects):
    # usefull_data.append(extract_usefull_data(item))
    try:
        item_data = restored_objects[key]
        extracted_data = extract_usefull_data(item_data)
        extracted_data["correlation_id"] = key
        checkp_name = extracted_data['webui_config']['override_settings']['sd_model_checkpoint']
        extracted_data["generation_type"] = f"{'sfw' if extracted_data['is_sfw'] else 'nsfw'}_{extracted_data['composition']}_{extracted_data['gender']}_{extracted_data['style']}_{checkp_name}"
        usefull_data.append(extracted_data)
    except Exception as e:
        print(e)
        lost_data_count += 1
        lost_data.append(restored_objects[key])
        continue
pd.DataFrame(usefull_data).to_csv('temp.csv', index=False)

  0%|          | 0/58441 [00:00<?, ?it/s]

In [5]:
# key = list(restored_objects.keys())[1269]
# extract_usefull_data(restored_objects[key])

In [6]:
# value = restored_objects[list(restored_objects.keys())[0]]
# find_key_in_nested_dict(value, 'height')

In [7]:
# usefull_df = pd.DataFrame(usefull_data)
usefull_df = pd.read_csv('temp.csv')
new_df = usefull_df.loc[usefull_df.result_image.apply(is_url_expired)]
# usefull_df['time_stamp'] = pd.to_datetime(usefull_df['time_stamp'])
# usefull_df = usefull_df.dropna(subset='ip_adapter_image')
# # Сортировка DataFrame по столбцу time_stamp в порядке убывания
# usefull_df = usefull_df.sort_values(by='time_stamp', ascending=False)

# usefull_df.head()

In [9]:
usefull_df.checkpoint_name.value_counts()

checkpoint_name
clampdxl_v30                    25236
dreamshaperXL_v21TurboDPMSDE    21266
natvisNaturalVision_v10          8439
realisticVisionV51_v51VAE        3420
realDream_sdxlPony11               80
Name: count, dtype: int64

### Extract data for new realistic checkpoint

In [10]:
# def remove_cjk_symbols(text):
#   """Removes CJK Unified Ideographs and similar symbols from a string.

#   Args:
#     text: The input string.

#   Returns:
#     The string with the specified symbols removed.
#   """

#   # Define the Unicode ranges to exclude
#   cjk_ranges = [
#       (0x3400, 0x4DBF),  # CJK Unified Ideographs Extension A
#       (0x4E00, 0x9FFF),  # CJK Unified Ideographs
#       (0x20000, 0x2A6DF),  # CJK Unified Ideographs Extension B
#       (0x2A700, 0x2B73F),  # CJK Unified Ideographs Extension C
#       (0x2B740, 0x2B81F),  # CJK Unified Ideographs Extension D
#       (0x2B820, 0x2CEAF),  # CJK Unified Ideographs Extension E
#       (0x2CEB0, 0x2EBEF),  # CJK Unified Ideographs Extension F
#       (0xF900, 0xFAFF),  # CJK Compatibility Ideographs
#       (0x2F800, 0x2FA1F)  # CJK Compatibility Ideographs Supplement
#   ]

#   # Filter the string, keeping only characters outside the specified ranges
#   filtered_text = "".join(char for char in text if not any(
#       start <= ord(char) <= end for start, end in cjk_ranges))

#   return filtered_text


# def extract_meta(img):
#     exif = { ExifTags.TAGS[k]: v for k, v in img._getexif().items() if k in ExifTags.TAGS }
#     exif["UserComment"] = exif["UserComment"].decode("utf-16-be")
#     return exif["UserComment"]


# # def parse_image_generation_string(input_string):
# #     """
# #     Parses a string containing image generation parameters and prompts,
# #     converting it into a dictionary.

# #     Args:
# #         input_string: The string to parse.

# #     Returns:
# #         A dictionary containing the parsed data.
# #     """

# #     data = {}

# #     # Split the string into positive prompt, negative prompt, and parameters
# #     parts = input_string.split("\nNegative prompt: ")
# #     if len(parts) != 2:
# #         raise ValueError("Invalid string format: Missing 'Negative prompt:' separator.")

# #     positive_prompt = parts[0].strip()
# #     data["positive_prompt"] = positive_prompt

# #     rest = parts[1].split("\nSteps: ")  # Split at "Steps:" to separate prompts from parameters
# #     if len(rest) != 2:
# #         raise ValueError("Invalid string format: Missing 'Steps:' separator.")

# #     negative_prompt = rest[0].strip()
# #     data["negative_prompt"] = negative_prompt

# #     parameters_str = "Steps: " + rest[1]  # Re-add "Steps:" to the parameters string
# #     parameters = parameters_str.split(", ")

# #     # Parse parameters
# #     for param in parameters:
# #         if ":" in param:
# #             key, value = param.split(":", 1)
# #             key = key.strip()
# #             value = value.strip()

# #             # Handle special cases like ControlNet, Lora hashes, and the remaining parameters
# #             if key.startswith("ControlNet"):
# #                 # Extract ControlNet details into a sub-dictionary
# #                 controlnet_data = {}
# #                 controlnet_parts = value.split(", ")
# #                 for part in controlnet_parts:
# #                     if ":" in part:
# #                         cn_key, cn_value = part.split(":", 1)
# #                         controlnet_data[cn_key.strip()] = cn_value.strip().strip('"')  # Remove quotes
# #                 data[key] = controlnet_data

# #             elif key == "Lora hashes":
# #                 # Extract Lora hashes into a dictionary
# #                 lora_data = {}
# #                 lora_parts = value.split(", ")
# #                 for part in lora_parts:
# #                     if ":" in part:
# #                         lora_name, lora_hash = part.split(":", 1)
# #                         lora_data[lora_name.strip()] = lora_hash.strip()
# #                 data[key] = lora_data

# #             else:
# #                 data[key] = value

# #     return data



In [ ]:
def load_single_img_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

def load_photos(sdwebui_request, load_dir: str):

    os.makedirs(load_dir, exist_ok=True)
    webui_img_path = f"{load_dir}/sdwebui_img.png"
    ip_adapter_img = None
    ip_adapter_img_path = None
    if not(pd.isna(sdwebui_request['ip_adapter_image'])):
        ip_adapter_img_path = f"{load_dir}/ip_adapter_img.png"
        ip_adapter_img = load_single_img_from_url(sdwebui_request["ip_adapter_image"])
        ip_adapter_img.save(ip_adapter_img_path)
    return {
        "webui_img_path": os.path.abspath(webui_img_path),
        "ip_adapter_img_path": os.path.abspath(ip_adapter_img_path) if ip_adapter_img_path else None,
    }



from datetime import datetime
import shutil


ban_checkpoints = ['realisticVisionV51_v51VAE']

def save_and_load_data(sample_name: str, ban_checkpoints: list[str], sampling_df: pd.DataFrame, k: int):
    sampling_df = sampling_df.loc[~sampling_df.checkpoint_name.isin(ban_checkpoints)]
    used_checkpoints = sampling_df.generation_type.value_counts().keys().tolist()
    dfs = [sample_generation_type(generation_type=generation_type, df=sampling_df, k=k) for generation_type in used_checkpoints]
    sampled_df = pd.concat(dfs)
    print(sampled_df.generation_type.value_counts())
    print(len(sampled_df[sampled_df.ip_adapter_image.isna()]), len(sampled_df[~sampled_df.ip_adapter_image.isna()]))
    sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)


    cur_date = datetime.now().strftime('%Y-%m-%d')
    exp_data_folder = f"photos_loaded/{sample_name}_load_data_{cur_date}"
    shutil.rmtree(exp_data_folder, ignore_errors=True)

    result_data = []
    for i in tqdm(range(len(sampled_df))):
        sdwebui_test = sampled_df.iloc[i]
        resp = load_photos(sdwebui_test, f"{exp_data_folder}/item_{i}")
        result_data.append(resp)


    sampled_df = pd.concat([sampled_df, pd.DataFrame(result_data)], axis=1)
    csv_path = f'dataframes/{sample_name}_{cur_date}.csv'
    sampled_df.to_csv(csv_path, index=False)
    print(csv_path)


def check_ip_adapter_image(item):
    return isinstance(item, str) and "cdn" in item

def sample_generation_type(generation_type, df, k=25):
    def sampled_if(item, k):
        return item.sample(n=k if len(item) > k else len(item), random_state=42)
    

    cp_df = df[df.generation_type == generation_type]
    ip_image_df = cp_df.loc[cp_df.ip_adapter_image.apply(check_ip_adapter_image)]
    without_ip_image_df = cp_df.loc[~cp_df.ip_adapter_image.apply(check_ip_adapter_image)]
    if len(ip_image_df) > 0:
        sampled_data = pd.concat([sampled_if(ip_image_df, k), sampled_if(without_ip_image_df, k)])
        return sampled_data
    else:
        return sampled_if(cp_df, 2*k)



In [12]:
# new_df['ip_adapter_image'] = new_df.ip_adapter_image.apply(lambda item: item[0] if item is None else None)

In [13]:
# # import os
# # import pandas as pd
# # import requests
# # from PIL import Image
# # from io import BytesIO
# # from datetime import datetime
# # import shutil
# # from tqdm import tqdm
# # from concurrent.futures import ThreadPoolExecutor


# def sample_generation_type(generation_type, df, k=25):
#     def sampled_if(item, k):
#         return item.sample(n=k if len(item) > k else len(item), random_state=42)
    

#     cp_df = df[df.generation_type == generation_type]
#     ip_image_df = cp_df.loc[cp_df.ip_adapter_image.apply(check_ip_adapter_image)]
#     without_ip_image_df = cp_df.loc[~cp_df.ip_adapter_image.apply(check_ip_adapter_image)]
#     if len(ip_image_df) > 0:
#         sampled_data = pd.concat([sampled_if(ip_image_df, k), sampled_if(without_ip_image_df, k)])
#         return sampled_data
#     else:
#         return sampled_if(cp_df, 2*k)

# # # Function to check IP adapter image validity
# # def check_ip_adapter_image(item):
# #     return isinstance(item, str) and "cdn" in item


# # # Function to sample generation types
# # def sample_generation_type(generation_type, df, k=25):
# #     def sampled_if(item, k):
# #         return item.sample(n=k if len(item) > k else len(item), random_state=42)

# #     cp_df = df[df.generation_type == generation_type]
# #     ip_image_df = cp_df[cp_df.ip_adapter_image.apply(check_ip_adapter_image)]
# #     without_ip_image_df = cp_df[~cp_df.ip_adapter_image.apply(check_ip_adapter_image)]
    
# #     if len(ip_image_df) > 0:
# #         sampled_data = pd.concat([sampled_if(ip_image_df, k), sampled_if(without_ip_image_df, k)])
# #         return sampled_data
# #     else:
# #         return sampled_if(cp_df, 2 * k)


# # # Optimized function to load a single image from a URL
# # def load_single_img_from_url(url):
# #     try:
# #         response = requests.get(url, stream=True, timeout=10)  # Use streaming and set timeout
# #         response.raise_for_status()  # Raise error for HTTP issues
# #         return Image.open(BytesIO(response.content))
# #     except Exception as e:
# #         print(f"Failed to load image from URL {url}: {e}")
# #         return None

# # # Optimized function to load photos
# # def load_photos(sdwebui_request, load_dir):
# #     """Loads images from the provided URLs and saves them to the specified directory."""
# #     os.makedirs(load_dir, exist_ok=True)  # Ensure the directory exists

# #     # Load the main web UI image
# #     sdwebui_img = load_single_img_from_url(sdwebui_request.get("result_image"))
# #     if sdwebui_img is None:  # Handle failures gracefully
# #         return {
# #             "positive_prompt": None,
# #             "negative_prompt": None,
# #             "webui_img_path": None,
# #             "ip_adapter_img_path": None,
# #         }

# #     webui_img_path = os.path.join(load_dir, "sdwebui_img.png")
# #     sdwebui_img.save(webui_img_path)  # Save the main image

# #     # Load the IP adapter image if available
# #     ip_adapter_img_path = None
# #     ip_adapter_image_url = sdwebui_request.get("ip_adapter_image")
# #     if ip_adapter_image_url and not pd.isna(ip_adapter_image_url):  # Check for valid URL
# #         ip_adapter_img = load_single_img_from_url(ip_adapter_image_url)
# #         if ip_adapter_img:
# #             ip_adapter_img_path = os.path.join(load_dir, "ip_adapter_img.png")
# #             ip_adapter_img.save(ip_adapter_img_path)

# #     # Return paths to the saved images
# #     return {
# #         "webui_img_path": webui_img_path,
# #         "ip_adapter_img_path": ip_adapter_img_path,
# #     }

# # # Optimized function to save and load data
# # def save_and_load_data(sample_name, ban_checkpoints, sampling_df, k):
# #     """Filters, samples, and processes data, saving results to disk."""
# #     # Filter out banned checkpoints
# #     sampling_df = sampling_df[~sampling_df["checkpoint_name"].isin(ban_checkpoints)]

# #     # Get unique generation types and sample k items per type
# #     used_checkpoints = sampling_df["generation_type"].unique()
# #     sampled_dfs = [
# #         sample_generation_type(generation_type, sampling_df, k) for generation_type in used_checkpoints
# #     ]
# #     sampled_df = pd.concat(sampled_dfs).sample(frac=1, random_state=42).reset_index(drop=True)

# #     # Prepare the output folder
# #     cur_date = datetime.now().strftime('%Y-%m-%d')
# #     exp_data_folder = f"img_data/load_data_{cur_date}"
# #     shutil.rmtree(exp_data_folder, ignore_errors=True)

# #     # Process the data in parallel
# #     result_data = []
# #     os.makedirs(exp_data_folder, exist_ok=True)
# #     with ThreadPoolExecutor() as executor:
# #         # Submit tasks for image loading and saving
# #         futures = [
# #             executor.submit(load_photos, sampled_df.iloc[i], f"{exp_data_folder}/item_{i}")
# #             for i in range(len(sampled_df))
# #         ]
# #         for future in tqdm(futures, desc="Processing Images"):
# #             result_data.append(future.result())

# #     # Combine results and save to CSV
# #     result_df = pd.DataFrame(result_data)
# #     final_df = pd.concat([sampled_df, result_df], axis=1)
# #     output_file = f"dataframes/{sample_name}_{cur_date}.csv"
# #     final_df.to_csv(output_file, index=False)
# #     print(f"Data saved to {output_file}")

In [14]:
# ban_checkpoints = ['realisticVisionV51_v51VAE']
# save_and_load_data('inference', ban_checkpoints, new_df, 10)

In [15]:
ban_checkpoints = ['realisticVisionV51_v51VAE', 'clampdxl_v30', 'dreamshaperXL_v21TurboDPMSDE', 'realDream_sdxlPony11']
save_and_load_data('natvis_solo_test', ban_checkpoints, new_df, 100)

generation_type
nsfw_full_None_simple_natvisNaturalVision_v10      200
nsfw_custom_None_simple_natvisNaturalVision_v10    200
Name: count, dtype: int64
200 200


  0%|          | 0/400 [00:00<?, ?it/s]

dataframes/natvis_solo_test_2024-12-06.csv
